In [1]:
!pip install chart_studio

import numpy as np 
import pandas as pd
import os

import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline
from pylab import rcParams
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
import statsmodels.api as sm

import math
import random

  Created wheel for retrying: filename=retrying-1.3.3-cp37-none-any.whl size=11435 sha256=3109e1c278289df0d07cc3b226e12ed863a94b8435f9229f4055b459e1fe1968
  Stored in directory: C:\Users\1\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [2]:
%%time
train = pd.read_csv('train.csv')
weather_train = pd.read_csv('weather_train.csv')
test = pd.read_csv('test.csv')
weather_test = pd.read_csv('weather_test.csv')
building_meta = pd.read_csv('building_metadata.csv')

Wall time: 4min 15s


In [3]:
train.head(20)

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0
5,5,0,2016-01-01 00:00:00,0.0
6,6,0,2016-01-01 00:00:00,0.0
7,7,0,2016-01-01 00:00:00,0.0
8,8,0,2016-01-01 00:00:00,0.0
9,9,0,2016-01-01 00:00:00,0.0


In [4]:
weather_train.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [5]:
test.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00


In [6]:
weather_test.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6


In [7]:
building_meta.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [8]:
#merging
train = train.merge(building_meta, on="building_id")

In [9]:
train = train.merge(weather_train, on=["site_id", "timestamp"])

In [10]:
sample = train.sample(20, random_state=0)
sample

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
2217631,255,3,2016-03-26 07:00:00,8.7921,2,Education,49078,1914.0,NaN,21.7,4.0,-6.7,0.0,1004.8,280.0,1.5
14843430,1209,0,2016-08-13 19:00:00,236.1380,13,Healthcare,115788,NaN,NaN,25.6,NaN,15.6,0.0,1012.9,320.0,5.7
17098627,1250,0,2016-06-27 17:00:00,664.5210,14,Education,232139,NaN,NaN,27.8,0.0,13.9,0.0,1016.9,190.0,3.6
10198113,888,2,2016-03-16 07:00:00,314.1500,9,Entertainment/public assembly,137522,NaN,NaN,19.4,0.0,6.1,0.0,1013.4,NaN,1.5
15314118,1102,2,2016-10-16 08:00:00,2609.3800,13,Office,87187,NaN,NaN,8.3,2.0,6.7,0.0,1011.6,0.0,0.0
15639128,1119,0,2016-11-29 04:00:00,52.0000,13,Office,119509,NaN,NaN,3.9,4.0,-0.6,0.0,982.4,230.0,2.6
8677154,765,0,2016-12-08 09:00:00,155.3670,6,Entertainment/public assembly,58562,NaN,NaN,2.2,NaN,0.0,0.0,1016.4,0.0,0.0
13837181,1150,1,2016-03-30 23:00:00,0.0000,13,Office,196043,NaN,NaN,7.8,NaN,4.4,0.0,1002.1,20.0,5.1
3671275,246,0,2016-10-22 10:00:00,351.8000,2,Office,202435,1967.0,NaN,25.0,0.0,5.6,0.0,1009.2,110.0,2.6
17242887,1244,0,2016-07-18 19:00:00,113.6400,14,Public services,95526,NaN,NaN,33.3,0.0,17.8,0.0,1015.7,250.0,5.1


In [11]:
train['primary_use'] = pd.Categorical(train['primary_use'])
train['primary_use'] = train['primary_use'].cat.codes
train.head(10)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,0,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,0,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,0,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,0,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,0,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
5,5,0,2016-01-01 00:00:00,0.0,0,0,8000,2000.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
6,6,0,2016-01-01 00:00:00,0.0,0,4,27926,1981.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
7,7,0,2016-01-01 00:00:00,0.0,0,0,121074,1989.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
8,8,0,2016-01-01 00:00:00,0.0,0,0,60809,2003.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
9,9,0,2016-01-01 00:00:00,0.0,0,6,27000,2010.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


In [13]:
#checking NaNs
100 * train.isnull().sum() / len(train)

building_id            0.000000
meter                  0.000000
timestamp              0.000000
meter_reading          0.000000
site_id                0.000000
primary_use            0.000000
square_feet            0.000000
year_built            60.188531
floor_count           82.631315
air_temperature        0.030623
cloud_coverage        43.401776
dew_temperature        0.047924
precip_depth_1_hr     18.178475
sea_level_pressure     5.670259
wind_direction         6.750371
wind_speed             0.264245
dtype: float64

In [14]:
del train['floor_count']
del train['year_built']

In [16]:
#preprocessing
def extract_data(building_id):
    return train[train['building_id'] == building_id]

extracted_df = {}
for i in range(10):
    df = extract_data(i)
    sea_lev_pressure_mean = df['sea_level_pressure'].mean()
    dew_temperature_mean = df['dew_temperature'].mean()
    air_temperature_mean = df['air_temperature'].mean()
    precip_depth_1_hr_mean = df['precip_depth_1_hr'].mean()
    wind_speed_mean = df['wind_speed'].median()

    df['meter'] = pd.to_numeric(df['meter'], errors='coerce').fillna(0).astype(np.float32)
    df['meter_reading'] = pd.to_numeric(df['meter_reading'], errors='coerce').fillna(0).astype(np.float32)
    df['site_id'] = pd.to_numeric(df['site_id'], errors='coerce').fillna(0).astype(np.float32)
    df['primary_use'] = pd.to_numeric(df['primary_use'], errors='coerce').fillna(0).astype(np.float32)
    df['square_feet'] = pd.to_numeric(df['square_feet'], errors='coerce').fillna(0).astype(np.float32)
    
    df['air_temperature'] = df['air_temperature'].fillna(-1)
    df['cloud_coverage'] = df['cloud_coverage'].fillna(-1)
    df['sea_level_pressure'] = df['sea_level_pressure'].fillna(sea_lev_pressure_mean)
    df['dew_temperature'] = df['dew_temperature'].fillna(dew_temperature_mean)
    df['air_temperature'] = df['air_temperature'].fillna(air_temperature_mean)
    df['precip_depth_1_hr'] = df['precip_depth_1_hr'].fillna(precip_depth_1_hr_mean)
    df['wind_direction'] = df['wind_direction'].apply(lambda x: random.random() * 360.0 if (math.isnan(x)) else x)
    df['wind_speed'] = df['wind_speed'].fillna(wind_speed_mean).astype(np.float32)
    extracted_df[i] = df
#dictionary 'extracted_df' contains data for buildings_ids in range [0, 10) which have to be reshaped in order to feed them into LSTM neural network    
extracted_df[4]

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
4,4,0.0,2016-01-01 00:00:00,0.000000,0.0,0.0,116607.0,25.0,6.0,20.0,1.382557,1019.7,0.0,0.0
107,4,0.0,2016-01-01 01:00:00,0.000000,0.0,0.0,116607.0,24.4,-1.0,21.1,-1.000000,1020.2,70.0,1.5
210,4,0.0,2016-01-01 02:00:00,0.000000,0.0,0.0,116607.0,22.8,2.0,21.1,0.000000,1020.2,0.0,0.0
313,4,0.0,2016-01-01 03:00:00,0.000000,0.0,0.0,116607.0,21.1,2.0,20.6,0.000000,1020.1,0.0,0.0
416,4,0.0,2016-01-01 04:00:00,0.000000,0.0,0.0,116607.0,20.0,2.0,20.0,-1.000000,1020.0,250.0,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1076026,4,0.0,2016-12-31 19:00:00,1150.800049,0.0,0.0,116607.0,22.8,-1.0,10.0,0.000000,1021.7,140.0,5.7
1076154,4,0.0,2016-12-31 20:00:00,1197.890015,0.0,0.0,116607.0,23.3,-1.0,8.9,0.000000,1021.0,170.0,4.1
1076282,4,0.0,2016-12-31 21:00:00,1147.380005,0.0,0.0,116607.0,23.3,-1.0,10.0,0.000000,1021.1,150.0,4.1
1076410,4,0.0,2016-12-31 22:00:00,1130.319946,0.0,0.0,116607.0,22.8,-1.0,10.0,0.000000,1021.1,160.0,3.1


In [ ]:
#building simple LSTM network

In [17]:
def input_function(x_tr, y_tr, x_d, y_d, x_t, y_t, bs_tr):
    t_d = tf.data.Dataset.from_tensor_slices((x_tr, y_tr))
    t_d = t_d.cache().batch(bs_tr).repeat()
    d_d = tf.data.Dataset.from_tensor_slices((x_d, y_d))
    d_d = d_d.batch(bs_tr).repeat()
    v_d = tf.data.Dataset.from_tensor_slices((x_t, y_t))
    v_d = v_d.batch(1).repeat()
    return t_d, d_d, v_d

In [18]:
def split_data(ds, tar, st_ind, en_ind, hs,
               ts, step):
    data = []
    lab = []

    st_ind = st_ind + hs
    if en_ind is None:
        en_ind = len(ds) - ts

    for i in range(st_ind, en_ind):
        indices = range(i - hs, i, step)
        data.append(ds[indices])
        lab.append(tar[i + ts])

    return np.array(data), np.array(lab)

In [19]:
def prepare_data(data, hn, tv_split,
                 ve_split, tn, step=1, bs=64):
    ds = {}

    for j in range(10):
        x_tr, y_tr = split_data(data[j], data[j][:, 3], 0,
                                          tv_split, hn,
                                          tn, step)
        x_d, y_d = split_data(data[j], data[j][:, 3],
                                      tv_split, ve_split, hn,
                                      tn, step)
        x_t, y_t = split_data(data[j], data[j][:, 3],
                                        ve_split, None, hn,
                                        tn, step)
        t_d, d_d, v_d = input_function(x_tr, y_tr, x_d, y_d, x_t, y_t, bs)
        ds[j] = {}
        ds[j]['train'] = t_d
        ds[j]['dev'] = d_d
        ds[j]['test'] = v_d

    inp_d = x_tr.shape[-2:]

    return inp_d, ds

In [20]:
def scale_data(data_dict, fc, fc_n):
    td = {}
    norm_param = {}

    for i in range(10):
        norm_param[i] = {}
        features = data_dict[i][fc]
        features.index = data_dict[i]['timestamp']
        features.head()
        for j in fc_n:
            data_mean = features[j].mean(axis=0)
            data_std = features[j].std(axis=0)
            features[j] = (features[j] - data_mean) / data_std
        td[i] = features.values
    
    return td, norm_param
        


In [21]:
def fit_model(ds, model, eval_int, epochs=10, trg='meter'):
    tm_histories = {}
    for i in range(10):
        print("Fitting to %s" % i)
        tm_histories[i] = model.fit(ds[i]["train"],
                                   validation_data=ds[i]["dev"],
                                   steps_per_epoch=eval_int,
                                   epochs=epochs,
                                   validation_steps=1).history
    return model, tm_histories

In [22]:
fc = ['meter', 'meter_reading', 'site_id',
      'primary_use', 'air_temperature', 'cloud_coverage',
      'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
      'wind_direction', 'wind_speed']

fc_n = ['meter_reading','air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed']

tv_split = int(len(extracted_df[4])* 0.7)
ve_split = int(len(extracted_df[4])* 0.85)
past = 30
eval_int = 1

td_set, norm_params = scale_data(extracted_df, fc, fc_n)
inp_d, ds = prepare_data(td_set, past, tv_split, ve_split, 1, step=1, bs=32)

In [23]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units=30,
                               return_sequences=True,
                               input_shape=inp_d))
model.add(tf.keras.layers.LSTM(units=32, return_sequences=True))
model.add(tf.keras.layers.Dropout(0.8))
model.add(tf.keras.layers.LSTM(units=16, return_sequences=True))
model.add(tf.keras.layers.Dropout(0.8))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer=tf.keras.optimizers.Adam(clipvalue=1.0), loss='mse')

In [24]:
model, hist_dict = fit_model(ds, model, eval_int, 500)

Fitting to 0
Train for 1 steps, validate for 1 steps
Epoch 1/500
1/1 [==============================] - 22s 22s/step - loss: 0.0281 - val_loss: 0.0016
Epoch 2/500
1/1 [==============================] - 6s 6s/step - loss: 0.0471 - val_loss: 0.0011
Epoch 3/500
1/1 [==============================] - 0s 156ms/step - loss: 0.0548 - val_loss: 8.1049e-04
Epoch 4/500
1/1 [==============================] - 0s 176ms/step - loss: 0.0328 - val_loss: 6.1306e-04
Epoch 5/500
1/1 [==============================] - 0s 216ms/step - loss: 0.0245 - val_loss: 5.1158e-04
Epoch 6/500
1/1 [==============================] - 0s 189ms/step - loss: 0.0191 - val_loss: 4.4148e-04
Epoch 7/500
1/1 [==============================] - 1s 731ms/step - loss: 0.0251 - val_loss: 4.0323e-04
Epoch 8/500
1/1 [==============================] - 0s 174ms/step - loss: 0.0218 - val_loss: 3.8798e-04
Epoch 9/500
1/1 [==============================] - 1s 1s/step - loss: 0.0227 - val_loss: 3.8073e-04
Epoch 10/500
1/1 [=================

Epoch 75/500
1/1 [==============================] - 0s 199ms/step - loss: 0.0021 - val_loss: 1.9890e-04
Epoch 76/500
1/1 [==============================] - 0s 197ms/step - loss: 0.0018 - val_loss: 1.9414e-04
Epoch 77/500
1/1 [==============================] - 0s 182ms/step - loss: 0.0017 - val_loss: 1.8946e-04
Epoch 78/500
1/1 [==============================] - 0s 182ms/step - loss: 0.0021 - val_loss: 1.8410e-04
Epoch 79/500
1/1 [==============================] - 0s 189ms/step - loss: 0.0012 - val_loss: 1.7856e-04
Epoch 80/500
1/1 [==============================] - 0s 189ms/step - loss: 0.0037 - val_loss: 1.7308e-04
Epoch 81/500
1/1 [==============================] - 0s 215ms/step - loss: 0.0026 - val_loss: 1.6676e-04
Epoch 82/500
1/1 [==============================] - 0s 250ms/step - loss: 0.0023 - val_loss: 1.6066e-04
Epoch 83/500
1/1 [==============================] - 0s 176ms/step - loss: 0.0018 - val_loss: 1.5536e-04
Epoch 84/500
1/1 [==============================] - 0s 187ms/ste

Epoch 153/500
1/1 [==============================] - 0s 179ms/step - loss: 0.0011 - val_loss: 2.4159e-05
Epoch 154/500
1/1 [==============================] - 0s 170ms/step - loss: 0.0011 - val_loss: 2.2537e-05
Epoch 155/500
1/1 [==============================] - 0s 162ms/step - loss: 7.8919e-04 - val_loss: 2.0936e-05
Epoch 156/500
1/1 [==============================] - 0s 184ms/step - loss: 6.5215e-04 - val_loss: 1.9504e-05
Epoch 157/500
1/1 [==============================] - 0s 189ms/step - loss: 8.4115e-04 - val_loss: 1.8570e-05
Epoch 158/500
1/1 [==============================] - 0s 199ms/step - loss: 0.0013 - val_loss: 1.8310e-05
Epoch 159/500
1/1 [==============================] - 0s 175ms/step - loss: 0.0011 - val_loss: 1.7659e-05
Epoch 160/500
1/1 [==============================] - 0s 181ms/step - loss: 6.1220e-04 - val_loss: 1.6835e-05
Epoch 161/500
1/1 [==============================] - 0s 171ms/step - loss: 5.1818e-04 - val_loss: 1.6111e-05
Epoch 162/500
1/1 [================

Epoch 229/500
1/1 [==============================] - 0s 176ms/step - loss: 8.4888e-04 - val_loss: 6.5517e-06
Epoch 230/500
1/1 [==============================] - 0s 178ms/step - loss: 6.1949e-04 - val_loss: 6.3748e-06
Epoch 231/500
1/1 [==============================] - 0s 174ms/step - loss: 4.3586e-04 - val_loss: 6.1687e-06
Epoch 232/500
1/1 [==============================] - 0s 171ms/step - loss: 4.0255e-04 - val_loss: 5.9504e-06
Epoch 233/500
1/1 [==============================] - 0s 168ms/step - loss: 6.3995e-04 - val_loss: 5.8068e-06
Epoch 234/500
1/1 [==============================] - 0s 168ms/step - loss: 4.0099e-04 - val_loss: 5.7106e-06
Epoch 235/500
1/1 [==============================] - 0s 196ms/step - loss: 4.1515e-04 - val_loss: 5.6422e-06
Epoch 236/500
1/1 [==============================] - 0s 165ms/step - loss: 3.9665e-04 - val_loss: 5.5715e-06
Epoch 237/500
1/1 [==============================] - 0s 186ms/step - loss: 5.7452e-04 - val_loss: 5.5062e-06
Epoch 238/500
1/1 [

Epoch 305/500
1/1 [==============================] - 0s 204ms/step - loss: 2.6706e-04 - val_loss: 6.1409e-06
Epoch 306/500
1/1 [==============================] - 0s 237ms/step - loss: 5.6073e-04 - val_loss: 5.9388e-06
Epoch 307/500
1/1 [==============================] - 0s 191ms/step - loss: 3.3735e-04 - val_loss: 5.9008e-06
Epoch 308/500
1/1 [==============================] - 0s 174ms/step - loss: 3.4396e-04 - val_loss: 6.0164e-06
Epoch 309/500
1/1 [==============================] - 0s 199ms/step - loss: 2.9141e-04 - val_loss: 6.1059e-06
Epoch 310/500
1/1 [==============================] - 0s 175ms/step - loss: 6.4730e-04 - val_loss: 5.7805e-06
Epoch 311/500
1/1 [==============================] - 0s 225ms/step - loss: 7.2386e-04 - val_loss: 5.0991e-06
Epoch 312/500
1/1 [==============================] - 0s 179ms/step - loss: 4.4469e-04 - val_loss: 4.6188e-06
Epoch 313/500
1/1 [==============================] - 0s 181ms/step - loss: 2.7312e-04 - val_loss: 4.3512e-06
Epoch 314/500
1/1 [

...

Epoch 171/500
1/1 [==============================] - 0s 198ms/step - loss: 4.9728 - val_loss: 0.8826
Epoch 172/500
1/1 [==============================] - 0s 192ms/step - loss: 5.0903 - val_loss: 0.8827
Epoch 173/500
1/1 [==============================] - 0s 253ms/step - loss: 5.3852 - val_loss: 0.8828
Epoch 174/500
1/1 [==============================] - 0s 218ms/step - loss: 5.2236 - val_loss: 0.8836
Epoch 175/500
1/1 [==============================] - 0s 214ms/step - loss: 5.6592 - val_loss: 0.8844
Epoch 176/500
1/1 [==============================] - 0s 225ms/step - loss: 5.2575 - val_loss: 0.8846
Epoch 177/500
1/1 [==============================] - 0s 189ms/step - loss: 5.5446 - val_loss: 0.8859
Epoch 178/500
1/1 [==============================] - 0s 221ms/step - loss: 5.3947 - val_loss: 0.8874
Epoch 179/500
1/1 [==============================] - 0s 237ms/step - loss: 5.6563 - val_loss: 0.8888
Epoch 180/500
1/1 [==============================] - 0s 185ms/step - loss: 4.9587 - val_los

1/1 [==============================] - 0s 233ms/step - loss: 5.3359 - val_loss: 1.0010
Epoch 253/500
1/1 [==============================] - 0s 214ms/step - loss: 5.1554 - val_loss: 1.0024
Epoch 254/500
1/1 [==============================] - 0s 165ms/step - loss: 5.3577 - val_loss: 1.0041
Epoch 255/500
1/1 [==============================] - 0s 217ms/step - loss: 5.1739 - val_loss: 1.0064
Epoch 256/500
1/1 [==============================] - 0s 169ms/step - loss: 5.2573 - val_loss: 1.0080
Epoch 257/500
1/1 [==============================] - 0s 188ms/step - loss: 5.0262 - val_loss: 1.0092
Epoch 258/500
1/1 [==============================] - 0s 194ms/step - loss: 5.3965 - val_loss: 1.0101
Epoch 259/500
1/1 [==============================] - 0s 208ms/step - loss: 5.1093 - val_loss: 1.0110
Epoch 260/500
1/1 [==============================] - 0s 259ms/step - loss: 5.2442 - val_loss: 1.0117
Epoch 261/500
1/1 [==============================] - 0s 192ms/step - loss: 5.2709 - val_loss: 1.0118
Epoc

Epoch 334/500
1/1 [==============================] - 0s 193ms/step - loss: 4.7037 - val_loss: 0.9498
Epoch 335/500
1/1 [==============================] - 0s 195ms/step - loss: 5.0094 - val_loss: 0.9495
Epoch 336/500
1/1 [==============================] - 0s 193ms/step - loss: 4.8759 - val_loss: 0.9484
Epoch 337/500
1/1 [==============================] - 0s 178ms/step - loss: 4.8088 - val_loss: 0.9472
Epoch 338/500
1/1 [==============================] - 0s 219ms/step - loss: 5.3237 - val_loss: 0.9465
Epoch 339/500
1/1 [==============================] - 0s 199ms/step - loss: 4.8330 - val_loss: 0.9456
Epoch 340/500
1/1 [==============================] - 0s 196ms/step - loss: 4.8429 - val_loss: 0.9446
Epoch 341/500
1/1 [==============================] - 0s 201ms/step - loss: 5.0903 - val_loss: 0.9431
Epoch 342/500
1/1 [==============================] - 0s 200ms/step - loss: 4.8682 - val_loss: 0.9411
Epoch 343/500
1/1 [==============================] - 0s 191ms/step - loss: 5.1417 - val_los

1/1 [==============================] - 0s 224ms/step - loss: 5.1221 - val_loss: 0.9072
Epoch 416/500
1/1 [==============================] - 0s 163ms/step - loss: 4.8726 - val_loss: 0.9058
Epoch 417/500
1/1 [==============================] - 0s 191ms/step - loss: 4.8935 - val_loss: 0.9048
Epoch 418/500
1/1 [==============================] - 0s 172ms/step - loss: 4.8656 - val_loss: 0.9036
Epoch 419/500
1/1 [==============================] - 0s 190ms/step - loss: 5.2177 - val_loss: 0.9026
Epoch 420/500
1/1 [==============================] - 0s 196ms/step - loss: 4.7627 - val_loss: 0.9020
Epoch 421/500
1/1 [==============================] - 0s 169ms/step - loss: 4.7346 - val_loss: 0.9007
Epoch 422/500
1/1 [==============================] - 0s 193ms/step - loss: 4.7216 - val_loss: 0.8987
Epoch 423/500
1/1 [==============================] - 0s 165ms/step - loss: 4.6117 - val_loss: 0.8957
Epoch 424/500
1/1 [==============================] - 0s 191ms/step - loss: 4.8033 - val_loss: 0.8934
Epoc

Epoch 497/500
1/1 [==============================] - 0s 176ms/step - loss: 4.8028 - val_loss: 0.8946
Epoch 498/500
1/1 [==============================] - 0s 233ms/step - loss: 4.5889 - val_loss: 0.8927
Epoch 499/500
1/1 [==============================] - 0s 205ms/step - loss: 4.5116 - val_loss: 0.8911
Epoch 500/500
1/1 [==============================] - 0s 174ms/step - loss: 4.9427 - val_loss: 0.8894
